In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/SchwiftyNotebook_matrix_multiplication")' SchwiftyNotebook_matrix_multiplication

Installing packages:
	.package(path: "/home/ubuntu/workspace/fast-ai-swift/SchwiftyNotebook_matrix_multiplication")
		SchwiftyNotebook_matrix_multiplication
With SwiftPM flags: []
Working in: /tmp/tmp200oepli/swift-install
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no vers

In [2]:
//export
import Path
import TensorFlow
import SchwiftyNotebook_matrix_multiplication

In [3]:
let thingo = Tensor([-2, -1, 0, 1, 2])

In [4]:
max(0, thingo)

[0.0, 0.0, 0.0, 1.0, 2.0]


In [7]:
thingo.replacing(with: Tensor(zeros: thingo.shape), where: thingo .< 0)

[0.0, 0.0, 0.0, 1.0, 2.0]


In [8]:
time(repeating: 2){ relu(thingo) }

average: 0.1787995 ms,   min: 0.166213 ms,   max: 0.191386 ms


In [9]:
//export
public func reLU<T>(tensor: Tensor<T>) -> Tensor<T> where T : FloatingPoint, T: TensorFlowScalar {
    return max(tensor, 0)
}

In [10]:
//export
public func linearCombination<T>(inputs: Tensor<T>, weights: Tensor<T>, bias: Tensor<T>) -> Tensor<T> where T: FloatingPoint, T: TensorFlowScalar {
    return matmul(inputs, weights) + bias
}

# Normalization

In [11]:
let (xTrainingData, yTrainingData, xValidationData, yValidationData) = loadMNISTData(path: mnistPath)

In [12]:
let xTrainingDataNormalized = normalizeTensor(tensor: xTrainingData)
                                        .reshaped(to: [xTrainingData.shape[0], 784])
let xValidationDataNormalized = normalizeTensor(tensor: xValidationData)
                                        .reshaped(to: [xValidationData.shape[0], 784])

In [13]:
// export
public typealias TensorFloat=Tensor<Float>
public func assertNearZero(_ tensor: TensorFloat, _ threshold: Float = 1e-3){
    assert(tensor < threshold, "Expected \(tensor) to be less than \(threshold)")
}

In [14]:
assertNearZero(xTrainingDataNormalized.mean())
assertNearZero(xValidationDataNormalized.mean())
assertNearZero(1 - xTrainingDataNormalized.standardDeviation())
assertNearZero(1 - xValidationDataNormalized.standardDeviation())

In [15]:
let (numberOfImages, numberOfPixels) = (xTrainingDataNormalized.shape[0], xTrainingDataNormalized.shape[1])
let numberOfClasses = 10
let layerOutput = 50
print(numberOfImages, numberOfPixels, numberOfClasses)

60000 784 10


# Initialization

## From Hand

In [16]:
let parameterLayerOne = TensorFloat(randomNormal: [numberOfPixels, layerOutput]) / sqrt(Float(numberOfPixels))
let parameterLayerTwo = TensorFloat(randomNormal: [layerOutput, 1]) / sqrt(Float(layerOutput))

In [17]:
assertNearZero(parameterLayerOne.mean())
assertNearZero(parameterLayerOne.standardDeviation() - 1 / sqrt(Float(numberOfPixels)))

In [18]:
let biasLayerOne = TensorFloat(zeros: [layerOutput])
let biasLayerTwo = TensorFloat(zeros: [1])

In [19]:
assertNearZero(biasLayerOne.mean())
assertNearZero(biasLayerOne.standardDeviation() - 1 / sqrt(Float(numberOfImages)))

In [20]:
print(xValidationDataNormalized.shape, parameterLayerOne.shape, biasLayerOne.shape)

[10000, 784] [784, 50] [50]


### Side Adventure: Timing

In [21]:
//export
import Dispatch

func getTimeUnit(_ nanoSeconds: Double) -> String {
    let powerOfTen = floor(log10(nanoSeconds))
    switch powerOfTen {
        case 1..<3:
            return "\(nanoSeconds) ns"
        case 3..<6:
            return "\(nanoSeconds/1e3) µs"
        case 6..<9: 
            return "\(nanoSeconds/1e6) ms"
        default: 
            return "\(nanoSeconds/1e9) s"
    }
}

// ⏰Time how long it takes to run the specified function, optionally taking
// the average across a number of repetitions.
public func withTime<T>(_ f: () -> T) -> T {
    let start = DispatchTime.now()
    let value = f()
    let end = DispatchTime.now()
    let nanoSeconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
    print("elapsed time: \(getTimeUnit(nanoSeconds))")
    return value
}

In [22]:
withTime{
    print("yeet")
}

yeet
elapsed time: 32.073 µs


In [23]:
let linearWomboCombo = time(repeating: 10) {
    withDevice(.gpu){
        linearCombination(inputs: xValidationDataNormalized, 
                                         weights: parameterLayerOne, 
                                         bias: biasLayerOne)
    }
}

average: 0.052803 ms,   min: 0.049133 ms,   max: 0.073722 ms


In [24]:
let cpuLinearWomboCombo = time(repeating: 10) {
    withDevice(.cpu){
        linearCombination(inputs: xValidationDataNormalized, 
                                         weights: parameterLayerOne, 
                                         bias: biasLayerOne)
    }
}

average: 16.0041841 ms,   min: 12.545927 ms,   max: 30.067697 ms


In [25]:
let defaultLinearWomboCombo = time(repeating: 10) {
    linearCombination(inputs: xValidationDataNormalized, 
                                     weights: parameterLayerOne, 
                                     bias: biasLayerOne)
}

average: 0.04406920000000001 ms,   min: 0.041844 ms,   max: 0.053992 ms


In [26]:
let thing = withTime { linearCombination(inputs: xValidationDataNormalized, 
                                         weights: parameterLayerOne, 
                                         bias: biasLayerOne) }

elapsed time: 131.739 µs


---

In [27]:
let linearWomboCombo = withTime { linearCombination(inputs: xValidationDataNormalized, 
                                         weights: parameterLayerOne, 
                                         bias: biasLayerOne) }

elapsed time: 160.33 µs


In [28]:
(linearWomboCombo.mean(), linearWomboCombo.standardDeviation())

▿ 2 elements
  - .0 : -0.05146276
  - .1 : 0.99336696


In [29]:
import Python
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

('inline', 'module://ipykernel.pylab.backend_inline')


In [30]:
public let plotter = Python.import("matplotlib.pyplot")

In [31]:
linearWomboCombo.shape

▿ [10000, 50]
  ▿ dimensions : 2 elements
    - 0 : 10000
    - 1 : 50


In [32]:
let activations = withTime { reLU(tensor: linearWomboCombo) }

elapsed time: 353.702683 ms


In [33]:
print(activations.mean(), activations.standardDeviation()) //relu zeros out all of the stuff below zero (so only ~half remain)

0.36869007 0.55238414


### Kaiming Initialization

In [34]:
let kaimingInitParamOne = TensorFloat(randomNormal: [numberOfPixels, layerOutput]) * (2.0/sqrt(Float(numberOfPixels)))

In [35]:
(kaimingInitParamOne.mean(), kaimingInitParamOne.standardDeviation())

▿ 2 elements
  - .0 : 0.00038751191
  - .1 : 0.07107573


In [36]:
let kaimingWomboCombo = withTime{ linearCombination(inputs: xValidationDataNormalized, 
                                          weights: kaimingInitParamOne, 
                                          bias: biasLayerOne ) }

elapsed time: 191.697 µs


In [37]:
(kaimingWomboCombo.mean(), kaimingWomboCombo.standardDeviation())

▿ 2 elements
  - .0 : 0.12336031
  - .1 : 2.0584419


In [38]:
let kaimingActivations = reLU(tensor: kaimingWomboCombo)

In [39]:
(kaimingActivations.mean(), kaimingActivations.standardDeviation())

▿ 2 elements
  - .0 : 0.8769652
  - .1 : 1.2709255


### Basic Model

In [40]:
let kaimingParamOne = TensorFloat(randomNormal: [numberOfPixels, layerOutput]) * (2.0/sqrt(Float(numberOfPixels)))
let kaimingParamTwo = TensorFloat(randomNormal: [layerOutput, 1]) * (2.0/sqrt(Float(layerOutput)))
let biasLayerOne = TensorFloat(zeros: [layerOutput])
let biasLayerTwo = TensorFloat(zeros: [1])

In [41]:
func basicBitch(_ inputTensor: TensorFloat) -> TensorFloat {
    let firstParams = withTime { linearCombination(inputs: inputTensor, 
                                                          weights: kaimingParamOne, 
                                                          bias: biasLayerOne) }
    let firstActivations = withTime { reLU(tensor: firstParams)}
    return withTime{ linearCombination(inputs: firstActivations, 
                             weights: kaimingParamTwo, 
                             bias: biasLayerTwo) }
}

In [42]:
let prediction = basicBitch(xValidationDataNormalized)

elapsed time: 177.381 µs
elapsed time: 353.703754 ms
elapsed time: 151.573 µs


🤔 Why is reLU so sloooow???

Turns out its just slow the first time...

In [43]:
for i in 1...10 {
 withTime { relu(kaimingWomboCombo)}   
}

elapsed time: 354.036533 ms
elapsed time: 211.414 µs
elapsed time: 142.85 µs
elapsed time: 121.612 µs
elapsed time: 112.358 µs
elapsed time: 124.002 µs
elapsed time: 108.954 µs
elapsed time: 110.809 µs
elapsed time: 112.624 µs
elapsed time: 109.226 µs


### Calculating Loss

In [44]:
(prediction.shape, yValidationData.shape)

▿ 2 elements
  ▿ .0 : [10000, 1]
    ▿ dimensions : 2 elements
      - 0 : 10000
      - 1 : 1
  ▿ .1 : [10000]
    ▿ dimensions : 1 element
      - 0 : 10000


In [45]:
prediction.squeezingShape(at: -1).shape

▿ [10000]
  ▿ dimensions : 1 element
    - 0 : 10000


In [46]:
(prediction.squeezingShape(at: -1) - TensorFloat(yValidationData)).squared().mean()

57.678993


In [47]:
func _meanSquaredError(_ y: TensorFloat, _ yHat: TensorFloat) -> TensorFloat {
    return (y - yHat.squeezingShape(at: [1])).squared().mean()
}

In [48]:
let trainingPredictions = basicBitch(xTrainingDataNormalized)

elapsed time: 194.516 µs
elapsed time: 353.416443 ms
elapsed time: 129.372 µs


In [49]:
(yTrainingData.shape, trainingPredictions.squeezingShape(at: [1]).shape)

▿ 2 elements
  ▿ .0 : [60000]
    ▿ dimensions : 1 element
      - 0 : 60000
  ▿ .1 : [60000]
    ▿ dimensions : 1 element
      - 0 : 60000


In [50]:
print(_meanSquaredError(TensorFloat(yValidationData), prediction))
_meanSquaredError(TensorFloat(yTrainingData), trainingPredictions)

57.678993


56.2724


# Auto Diff

In [51]:
for x in stride(from: 0.0, 
                through: 1, by: 0.1){
    print(cos(x), sin(x))
}

1.0 0.0
0.9950041652780258 0.09983341664682815
0.9800665778412416 0.19866933079506122
0.955336489125606 0.2955202066613396
0.9210609940028851 0.3894183423086505
0.8775825618903728 0.479425538604203
0.8253356149096782 0.5646424733950355
0.7648421872844884 0.6442176872376911
0.6967067093471654 0.7173560908995228
0.6216099682706644 0.7833269096274834
0.5403023058681398 0.8414709848078965


**Note:** `in` is the arrow if the closure

In [52]:
let mathyBoiGradientFunction = gradient { (x: Double) in x * 9 }

In [53]:
mathyBoiGradientFunction(69)

9.0


The derivative of $9x$ with respect to $x$ is just 9

In [54]:
let sineGraident = gradient { (x: Double) in sin(x)}
let cosineGraident = gradient { (x: Double) in cos(x)}

for x in stride(from: 0.0, 
                through: 1, by: 0.1){
    print(cosineGraident(x), sineGraident(x))
}

-0.0 1.0
-0.09983341664682815 0.9950041652780258
-0.19866933079506122 0.9800665778412416
-0.2955202066613396 0.955336489125606
-0.3894183423086505 0.9210609940028851
-0.479425538604203 0.8775825618903728
-0.5646424733950355 0.8253356149096782
-0.6442176872376911 0.7648421872844884
-0.7173560908995228 0.6967067093471654
-0.7833269096274834 0.6216099682706644
-0.8414709848078965 0.5403023058681398


The derivative of sin is cosine and the derivitive of cosine is -sine. 2cool4Me

### Basics Of Mean Squared Error

In [55]:
func tracer(functionName: String = #function) { //I'm already tracer
    print(functionName)
}

In [56]:
let closureDude = {
    tracer()
}

closureDude()

__lldb_expr_449


In [57]:
func imAlreadyTracer() {
    tracer()
    print("maybe I'll be tracer")
}

imAlreadyTracer()

imAlreadyTracer()
maybe I'll be tracer


In [58]:
let a = TensorFloat([[1, 2, 3],[4,5,6]])
a.shape

▿ [2, 3]
  ▿ dimensions : 2 elements
    - 0 : 2
    - 1 : 3


In [59]:
a.sum(squeezingAxes: [0, 1])

21.0


In [60]:
func squareTensor(_ x: TensorFloat) -> TensorFloat {
    tracer()
    return x * x
}

func 𝛁squareTensor(_ x: TensorFloat) -> TensorFloat {
    tracer()
    return 2 * x
}

func meanOfTensor(_ x: TensorFloat) -> TensorFloat {
    tracer()
    return x.sum(squeezingAxes: [0, 1]) / Float(x.shape[0])
}

// The derivative of the mean with respect to x is just 1/n because the mean is just sum(weights)/n
// cus the deriviative of x/2 with respect to x is just 1/2
func 𝛁meanOfTensor(_ x: TensorFloat) -> TensorFloat {
    tracer()
    return TensorFloat(ones: [1]) / Float(x.shape[0])
}


In [61]:
(prediction.mean(), meanOfTensor(prediction))

meanOfTensor(_:)


▿ 2 elements
  - .0 : -2.0996099
  - .1 : -2.0996099


In [62]:
//export
public func assertEquals(_ a: TensorFloat, _ b: TensorFloat) {
    // Check shapes match so broadcasting doesn't hide shape errors.
    assert(a.shape == b.shape)
    assertNearZero(a-b)
}

In [63]:
func meanSquareError(_ x: TensorFloat) -> TensorFloat {
    return meanOfTensor(squareTensor(x))
}

func 𝛁meanSquareError(_ x: TensorFloat) -> TensorFloat {
    return 𝛁meanOfTensor(squareTensor(x)) * 𝛁squareTensor(x)
}

let myGradientSquareError = 𝛁meanSquareError(prediction)
let fancyPantsLibraryGradient = gradient { (x: TensorFloat) in x.squared().mean()}(prediction
                                                                                  )

assertEquals(fancyPantsLibraryGradient, myGradientSquareError)

squareTensor(_:)
𝛁meanOfTensor(_:)
𝛁squareTensor(_:)


In [64]:
func square𝛁_Chain(x: TensorFloat, ddx: TensorFloat) -> TensorFloat {
    tracer()
    return 2 * x * ddx
}

func mean𝛁_Chain(x: TensorFloat, ddx: TensorFloat) -> TensorFloat {
    tracer()
    return (TensorFloat(ones: [1]) / Float(x.shape[0])) * ddx
}

In [65]:
func squareWomboCombo(_ x: TensorFloat) -> (value: TensorFloat, chain: (TensorFloat) -> TensorFloat){
    tracer()
    return (value: x * x, chain: { ddx in square𝛁_Chain(x: x, ddx: ddx) })
}

func meanWomboCombo(_ x: TensorFloat) -> (value: TensorFloat, chain: (TensorFloat) -> TensorFloat){
    tracer() // x is already squared at this point, no need to calculate things done to it.
    return (value: x.sum(squeezingAxes: [0, 1]) / Float(x.shape[0]), 
            chain: { ddx in mean𝛁_Chain(x: x, ddx: ddx )})
}

In [66]:
func meanSquaredErrorWithGradinent(_ x: TensorFloat) -> (value: TensorFloat, chain: (TensorFloat) -> TensorFloat) {
    tracer()
    let (squared, squaredChain) = squareWomboCombo(x)
    let (mean, meanChain) = meanWomboCombo(squared)
    
    return (value: mean, chain: { first in squaredChain(meanChain(first)) })
}

In [67]:
let (meanSquaredErr, mseGradient) = meanSquaredErrorWithGradinent(prediction)

meanSquaredErrorWithGradinent(_:)
squareWomboCombo(_:)
meanWomboCombo(_:)


In [68]:
assertEquals(prediction.squared().mean(), meanSquaredErr)

In [69]:
assertEquals(fancyPantsLibraryGradient, mseGradient(TensorFloat(ones: [1])))

mean𝛁_Chain(x:ddx:)
square𝛁_Chain(x:ddx:)


\o/

In [71]:
func reLUWomboCombo(_ inputTensor: TensorFloat) -> (value: TensorFloat,
                                                    chain: (TensorFloat) -> TensorFloat) {
    tracer()
    return (value: max(0.0, inputTensor), 
            chain: {ddxWomboCombo in max(0.0, ddxWomboCombo)})
}

womboCombo = inputs * parameters + bias

dWC/dp = inputs

dWC/db = 1


meanSquaredError(womboCombo(reLU(womboCombo())))

'meanSquaredError(womboCombo(reLU(womboCombo()))) * 'womboCombo(reLU(womboCombo())) * 'reLU(womboCombo()) * 'womboCombo()

In [ ]:
func womboCombo𝛁_Chain()

In [ ]:
func womboComboWomboCombo(_ inputTensor: TensorFloat, 
                          _ parameter: TensorFloat, 
                          _ bias: TensorFloat) -> (value: TensorFloat,
                                                   chain: (TensorFloat) -> TensorFloat) {
    tracer()
    return (value: linearCombination(inputs: inputTensor, 
                                     weights: parameter, 
                                     bias: bias), 
            chain: { ddxWomboCombo in  })
}